In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
from collections import Counter
from os.path import join as oj

import numpy as np
from matplotlib import pyplot as plt
plt.style.use('dark_background')
import pandas as pd
import pickle as pkl
from tqdm import tqdm
from style import *

# 'results/aug6_6/' has a lot of results
# aug6_combined_abd_tender - combined the different abdTender values
out_dir = 'results/aug6_6/' 

# compare all models

In [2]:
def load_results(out_dir):
    r = []
    for fname in tqdm(os.listdir(out_dir)):
        d = pkl.load(open(oj(out_dir, fname), 'rb'))
        d['param_name'] = fname[:-4]
        r.append(pd.Series(d))
    r = pd.concat(r, axis=1, sort=False).T.infer_objects()
    r = r.reindex(sorted(r.columns, reverse=True), axis=1) # sort the column names
    r = r.round(3)
    r = r.set_index('param_name')
    return r

results = load_results(out_dir)
# print(results.keys())

100%|██████████| 185/185 [00:02<00:00, 72.56it/s]


## look at prediction metrics

Most important thing is to have very low false negatives (very close to zero). After that, we also want to minimize the false positives.

In [12]:
# prev rule: sensitivity 0.97, specificity 0.425
r = results
# print(r.keys())
# r = r[[k for k in r if not 'std' in k]]
# r = r[[k for k in r if not '_test' in k]]
# r = r[[k for k in r if not '_f' in k]]
# r = r[r.index.str.contains('ros')] # only use random sampling
# r = r.sort_values(by=['balanced_accuracy'], ascending=False)
# r = r[r.model_type == 'grl']
# r = r[r.model_type == 'brl']
r = r[r['sensitivity_cv'] > 0.8] # what fraction of true pos do we return?
r = r[r['specificity_cv'] > 0.4] # what fraction of true negs do we return?

r = r.sort_values(by=['sensitivity_cv', 'specificity_cv'], ascending=False)
r = r[['sensitivity_cv', 'specificity_cv', # 'precision',
       'sensitivity_test1', 'specificity_test1',
       'sensitivity_test2', 'specificity_test2',
       'fn_cv', 'tn_cv', 'fp_cv', 'tp_cv',
       'precision_cv', 'roc_auc_cv', 
#        'f1_cv', 'balanced_accuracy_cv', 'accuracy_cv',
       'fn_test1', 'tn_test1', 'fp_test1', 'tp_test1',
       'fn_test2', 'tn_test2', 'fp_test2', 'tp_test2']]
# r.style.background_gradient(cmap='viridis', axis=None) # all values on same cmap
# r = r[r.index == 'logistic_select_lasso=27_ros=100']
# r.style.background_gradient(cmap='viridis', axis=0) # columns differently colored
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])
r

,sensitivity_cv,specificity_cv,sensitivity_test1,specificity_test1,sensitivity_test2,specificity_test2,fn_cv,tn_cv,fp_cv,tp_cv,precision_cv,roc_auc_cv,fn_test1,tn_test1,fp_test1,tp_test1,fn_test2,tn_test2,fp_test2,tp_test2
param_name,,,,,,,,,,,,,,,,,,,,
logistic_select_stab_lasso=6_sample_weights=500,0.969,0.522,0.877,0.513,0.984,0.507,4,4129,3774,126,0.032,0.898,9,2021,1917,64,1,1077,1049,61
logistic_select_stab_lasso=7_sample_weights=750,0.969,0.509,0.890,0.493,0.968,0.496,4,4026,3877,126,0.031,0.896,8,1943,1995,65,2,1055,1071,60
logistic_select_stab_lasso=6_sample_weights=750,0.969,0.504,0.904,0.484,0.984,0.476,4,3983,3920,126,0.031,0.897,7,1905,2033,66,1,1011,1115,61
logistic_select_stab_lasso=6_sample_weights=1000,0.969,0.504,0.904,0.484,0.984,0.476,4,3983,3920,126,0.031,0.897,7,1905,2033,66,1,1011,1115,61
logistic_select_stab_lasso=7_sample_weights=500,0.962,0.546,0.863,0.587,0.952,0.556,5,4312,3591,125,0.034,0.897,10,2312,1626,63,3,1182,944,59
logistic_select_stab_lasso=5_sample_weights=500,0.954,0.531,0.863,0.524,0.984,0.516,6,4193,3710,124,0.032,0.884,10,2063,1875,63,1,1097,1029,61
logistic_select_stab_lasso=10_sample_weights=1000,0.954,0.434,0.904,0.369,0.968,0.319,6,3428,4475,124,0.027,0.888,7,1455,2483,66,2,678,1448,60
logistic_select_stab_lasso=10_sample_weights=750,0.946,0.492,0.904,0.446,0.968,0.393,7,3886,4017,123,0.030,0.890,7,1756,2182,66,2,835,1291,60
logistic_select_stab_lasso=10_sample_weights=500,0.938,0.557,0.863,0.542,0.952,0.492,8,4399,3504,122,0.034,0.893,10,2135,1803,63,3,1046,1080,59


In [11]:
results.model_type.unique()

array(['brl', 'grl', 'dt', 'logistic'], dtype=object)

**look at feat importances**

In [37]:
best_model = 'logistic_select_stab_lasso=6_sample_weights=750' #logistic_select_rf=5_smote=5'
r = results
r = r[r.index.str.contains(best_model)].iloc[0]
d = pd.DataFrame.from_dict({
    'feats': r['feat_names_selected'],
    'imps': r['imps_best'],
})
d = d.iloc[np.argsort(d['imps'])[::-1]]
d = d.round(2)
d.fillna(0).style.background_gradient(cmap=cm) #, axis=0) #, subset=subset) # rows differently colored

,feats,imps
3,AbdTenderDegree_None,2.28
1,AbdTenderDegree_Mild,2.21
2,AbdTenderDegree_Moderate,2.13
4,AbdTenderDegree_Severe,1.89
5,AbdTrauma_or_SeatBeltSign_yes,0.83
6,DecrBreathSound_yes,0.39
0,AbdDistention_yes,0.35
9,VomitWretch_yes,0.07
8,Hypotension_yes,-0.01
7,GCSScore_Full_yes,-1.08


# misc analysis

In [ ]:
def count_common_feats(m_cv, model_type='tree'):
    if model_type == 'tree':
        feats_used = []
        for i in range(len(m_cv['estimator'])):
            m_fit = m_cv['estimator'][i]
            feats_used += list(ks[m_fit.feature_importances_ != 0])
    elif model_type == 'logistic':
        feats_used = []
        for i in range(len(m_cv['estimator'])):
            m_fit = m_cv['estimator'][i]
            num_feats = 5
            # get top num_feats features with biggest bsolute weights
            idxs = np.abs(m_fit.coef_).flatten().argsort()[-num_feats:][::-1]
            feats_used += list(ks[idxs])
    return sorted(dict(Counter(feats_used)).items(), key=lambda kv: kv[1], reverse=True)
        
count_common_feats(m_cv, model_type)

In [ ]:
# plot a tree
m_fit = m_cv['estimator'][0]
plt.figure(dpi=300)
plot_tree(m_fit, feature_names=ks)
plt.show()